# Import Library

In [1]:
import os
import csv
import pandas as pd
import shutil
#os.chdir('/mnt/c/Workspace/Ch3/wflow/')
#os.chdir('/mnt/d/')
os.chdir(os.path.join("D:\\GitHub\\wflow\\notebooks"))
import TempBuilder # importing custom built functions.

In [2]:
# Change back working directory to D
os.chdir(os.path.join("D:\\"))
os.getcwd()

'D:\\'

# Copy the instate/staticmaps corresponding to the model config that is to be used

In [3]:
wsName_list = ["san-diego_california_20331196_11023340",
               "fulton_georgia_2047963_02336410"
              ]
config = str("1km") #"90" or "1km"

In [4]:
#import logging
from datetime import datetime

#logging.basicConfig(filename='copy_log.log', level=logging.INFO, format='%(asctime)s - %(message)s')

config = str("1km") #"90" or "1km"

def copytree_and_overwrite(src, dst):
    if os.path.exists(dst):
        shutil.rmtree(dst)
    shutil.copytree(src, dst)

for n in wsName_list:
    static_origin = os.path.join("./wflow_models/", str(n+"/inputs_" + config), "staticmaps")
    instate_origin = os.path.join("./wflow_models/", str(n+"/inputs_" + config), "instate")
    static_dst = os.path.join("./wflow_models/", str(n), "staticmaps")
    instate_dst = os.path.join("./wflow_models/", str(n), "instate")
    
    copytree_and_overwrite(static_origin, static_dst)
    copytree_and_overwrite(instate_origin, instate_dst)
        
    # Write log entry
    log_file_path = os.path.join("./wflow_models/", str(n), "log_instate_static_copy.txt")
    with open(log_file_path, 'a') as log_file:
        log_entry = f'{datetime.now()} - Copied for: {n}, Config: {config}\n'
        log_file.write(log_entry)


# TempBuilder

In [7]:
wsName_list = ["san-diego_california_20331196_11023340",
               "fulton_georgia_2047963_02336410"
              ]
wsName_list = ["san-diego_california_20331196_11023340"]

#csv_file_path = os.path.join("./wflow_models/combined_samples.csv")

#df = pd.read_csv(csv_file_path)
#print(df)
wsName_list

['san-diego_california_20331196_11023340']

# Loop

In [8]:
identifiers = ['W', 'H', 'P']

model_directory = str("./wflow_models/")
intbl_directory = str("./wflow_models/" + "/intbl/")

for n in wsName_list:
    static_directory = str("./wflow_models/" + n + "/staticmaps/")
    temp_directory = str("./wflow_models/" + n + "/temp/")
    csv_file_path = os.path.join(str("./wflow_models/combined_samples_v4_Ks_"+ n + ".csv"))
    df = pd.read_csv(csv_file_path)

    print(str(os.path.abspath(temp_directory) +": processing"))

    
    for index, row in df.iterrows():
        
        print(f"\r{index + 1} / {len(df)}", end='    ', flush=True)
        temp_iter_directory = str(temp_directory+str(index))
        temp_iter_intbl_directory = temp_iter_directory + "/intbl/"
        
        #if not os.path.exists(temp_iter_directory):
        #    os.mkdir(temp_iter_directory)
            
        try:
            shutil.copytree(intbl_directory, temp_iter_intbl_directory, dirs_exist_ok=True)
        except Exception as e:
            print(f"Error occurred: {e}")
            
        TempBuilder.modify_and_save_ini(model_directory+"/wflow_tofuflex.ini", index, temp_iter_directory+"/wflow_tofuflex_"+str(index)+".ini")
        TempBuilder.modify_and_save_ini(model_directory+"/wflow_tofuflex_1km.ini", index, temp_iter_directory+"/wflow_tofuflex_1km_"+str(index)+".ini")
        TempBuilder.modify_and_save_ini(model_directory+"/wflow_tofuflex_ns.ini", index, temp_iter_directory+"/wflow_tofuflex_ns_"+str(index)+".ini")
        TempBuilder.modify_and_save_ini(model_directory+"/wflow_tofuflex_ns_1km.ini", index, temp_iter_directory+"/wflow_tofuflex_ns_1km_"+str(index)+".ini")
        TempBuilder.modify_and_save_ini(model_directory+"/wflow_topoflex_bm.ini", index, temp_iter_directory+"/wflow_topoflex_bm_"+str(index)+".ini")
        TempBuilder.modify_and_save_ini(model_directory+"/wflow_topoflex_bm_1km.ini", index, temp_iter_directory+"/wflow_topoflex_bm_1km_"+str(index)+".ini")
            
        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/betaW"+".tbl", row.iloc[0], temp_iter_intbl_directory+"/betaW"+".tbl")
        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/betaAP"+".tbl", row.iloc[0], temp_iter_intbl_directory+"/betaAP"+".tbl")
        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/betaH"+".tbl", row.iloc[1], temp_iter_intbl_directory+"/betaH"+".tbl")
        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/betaP"+".tbl", row.iloc[1], temp_iter_intbl_directory+"/betaP"+".tbl")
        
        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/DH"+".tbl", row.iloc[2], temp_iter_intbl_directory+"/DH"+".tbl")
        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/DP"+".tbl", row.iloc[3], temp_iter_intbl_directory+"/DP"+".tbl")

        # These three doesn't need to be separated by OG.
        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/KfW"+".tbl", row.iloc[4], temp_iter_intbl_directory+"/KfW"+".tbl")
        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/KfW_OG"+".tbl", row.iloc[4], temp_iter_intbl_directory+"/KfW_OG"+".tbl") # It is identical to KfW, but a place holder for topo_bm to make life easier
        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/KfaP"+".tbl", row.iloc[4], temp_iter_intbl_directory+"/KfaP"+".tbl")
        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/Kfimp"+".tbl", row.iloc[4], temp_iter_intbl_directory+"/Kfimp"+".tbl")

        # KfP_OG.tbl and KfH_OG.tbl is used for topo_bm, but process_Kf_files() will adjust it to KfP and KfH that goes into tofu model.
        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/KfP_OG"+".tbl", row.iloc[5], temp_iter_intbl_directory+"/KfP_OG"+".tbl")
        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/KfH_OG"+".tbl", row.iloc[5], temp_iter_intbl_directory+"/KfH_OG"+".tbl")

        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/sumaxW"+".tbl", row.iloc[6], temp_iter_intbl_directory+"/sumaxW"+".tbl")
        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/sumaxH"+".tbl", row.iloc[7], temp_iter_intbl_directory+"/sumaxH"+".tbl")
        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/sumaxP"+".tbl", row.iloc[7], temp_iter_intbl_directory+"/sumaxP"+".tbl")
        
        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/KsW"+".tbl", row.iloc[8], temp_iter_intbl_directory+"/KsW"+".tbl")
        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/KsH"+".tbl", row.iloc[8], temp_iter_intbl_directory+"/KsH"+".tbl")
        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/KsP"+".tbl", row.iloc[8], temp_iter_intbl_directory+"/KsP"+".tbl")
        
        TempBuilder.process_Kf_files(temp_iter_intbl_directory, identifiers, static_directory)
    print()



D:\wflow_models\san-diego_california_20331196_11023340\temp: processing
1000 / 1000    


# Just the ini files for quicker update

In [8]:
for n in wsName_list:
    static_directory = str("./wflow_models/" + n + "/staticmaps/")
    temp_directory = str("./wflow_models/" + n + "/temp/")
    print(str(os.path.abspath(temp_directory) +": processing just .ini files"))


    for index, row in df.iterrows():
        
        print(f"\r{index + 1} / {len(df)}", end='    ', flush=True)
        temp_iter_directory = str(temp_directory+str(index))
    
        TempBuilder.modify_and_save_ini(model_directory+"/wflow_tofuflex.ini", index, temp_iter_directory+"/wflow_tofuflex_"+str(index)+".ini")
        TempBuilder.modify_and_save_ini(model_directory+"/wflow_tofuflex_1km.ini", index, temp_iter_directory+"/wflow_tofuflex_1km_"+str(index)+".ini")
        TempBuilder.modify_and_save_ini(model_directory+"/wflow_tofuflex_ns.ini", index, temp_iter_directory+"/wflow_tofuflex_ns_"+str(index)+".ini")
        TempBuilder.modify_and_save_ini(model_directory+"/wflow_tofuflex_ns_1km.ini", index, temp_iter_directory+"/wflow_tofuflex_ns_1km_"+str(index)+".ini")
        TempBuilder.modify_and_save_ini(model_directory+"/wflow_topoflex_bm.ini", index, temp_iter_directory+"/wflow_topoflex_bm_"+str(index)+".ini")
        TempBuilder.modify_and_save_ini(model_directory+"/wflow_topoflex_bm_1km.ini", index, temp_iter_directory+"/wflow_topoflex_bm_1km_"+str(index)+".ini")
        
    print()

D:\wflow_models\san-diego_california_20331196_11023340\temp: processing just .ini files
200 / 200    
D:\wflow_models\fulton_georgia_2047963_02336410\temp: processing just .ini files
200 / 200    
